In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import hashlib
import datetime

In [48]:
# open the api.txt file
with open('api.txt', 'r') as f:
    pub_api = f.read()
# run the api key
with open('priv_api.txt', 'r') as f:
    priv_api = f.read()
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [49]:
def hash_params():
    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_api}{pub_api}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()
    return hashed_params


In [50]:
params = {'ts': timestamp, 'apikey': pub_api, 'hash': hash_params()}
response = requests.get("http://gateway.marvel.com/v1/public/comics", params=params)
print(response.status_code)

200


In [51]:
character_response = requests.get("http://gateway.marvel.com/v1/public/characters", params=params)
print(character_response.status_code)

200


In [56]:
if character_response.status_code == 200:
    results = character_response.text
    data = json.loads(results)
    character_df = pd.DataFrame(data)
character_df.columns
# df.to_csv('marvel.csv', index=False)

Index(['code', 'status', 'copyright', 'attributionText', 'attributionHTML',
       'etag', 'data'],
      dtype='object')

In [57]:
if response.status_code == 200:
    results = response.text
    data = json.loads(results)
    df = pd.DataFrame(data)
df.columns

Index(['code', 'status', 'copyright', 'attributionText', 'attributionHTML',
       'etag', 'data'],
      dtype='object')